In [35]:
import cProfile
from itertools import count

import numpy as np


def getSamplesPerRecode(numOfPoints):
    samplesPerRecord = (numOfPoints // 64) * 64
    if samplesPerRecord < numOfPoints:
        samplesPerRecord += 64
    return samplesPerRecord


def getExpArray(f_list, numOfPoints, weight=None, sampleRate=1e9):
    e = []
    t = np.arange(0, numOfPoints, 1) / sampleRate
    if weight is None:
        weight = np.ones(numOfPoints)
    for f in f_list:
        e.append(weight * np.exp(-1j * 2 * np.pi * f * t))
    return np.asarray(e).T


def aquireData():
    A, B = np.random.randn(64 * 1024), np.random.randn(64 * 1024)
    while True:
        yield A, B


recordsPerBuffer = 64
samplesPerRecord = 1024
maxlen = 10000
n = 1000
repeats = 0

e = getExpArray([50e6, 30e6], 1000)
fft = False
avg = False

def test():
    A, B = [], []

    for index, (chA, chB) in zip(count(), aquireData()):
        A_lst = chA.reshape((recordsPerBuffer, samplesPerRecord))
        B_lst = chB.reshape((recordsPerBuffer, samplesPerRecord))
        if fft:
            A_lst = (A_lst[:, :n]).dot(e)
            B_lst = (B_lst[:, :n]).dot(e)
        try:
            A.append(A_lst)
            B.append(B_lst)
        except:
            A, B = A_lst, B_lst
        if repeats == 0 and index * recordsPerBuffer >= maxlen:
            break
            
    A = np.asarray(A)
    B = np.asarray(B)
    
    A = A.flatten().reshape(A.shape[0]*A.shape[1], A.shape[2])
    B = B.flatten().reshape(B.shape[0]*B.shape[1], B.shape[2])
            
    if avg:
        return A.mean(axis=0), B.mean(axis=0)
    else:
        return A, B


cProfile.run('test()')

         805 function calls in 0.140 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      159    0.000    0.000    0.004    0.000 <ipython-input-35-d51686fffc1c>:24(aquireData)
        1    0.012    0.012    0.129    0.129 <ipython-input-35-d51686fffc1c>:40(test)
        1    0.012    0.012    0.140    0.140 <string>:1(<module>)
        2    0.000    0.000    0.054    0.027 _asarray.py:16(asarray)
        1    0.000    0.000    0.140    0.140 {built-in method builtins.exec}
        2    0.054    0.027    0.054    0.027 {built-in method numpy.array}
      316    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        2    0.057    0.029    0.057    0.029 {method 'flatten' of 'numpy.ndarray' objects}
        2    0.004    0.002    0.004    0.002 {method 'randn' of 'numpy.random.mtrand.RandomState' objects}
    

In [36]:
a,b = test()

In [37]:
a.shape

(10112, 1024)

In [17]:
x = np.array([
    [[1,2],[3,4],[5,6]],
    [[-1,-2],[-3,-4],[-5,-6]]
])

In [18]:
x.shape

(2, 3, 2)

In [29]:
x.flatten().reshape(2,6)

array([[ 1,  2,  3,  4,  5,  6],
       [-1, -2, -3, -4, -5, -6]])